<a href="https://colab.research.google.com/github/momo54/large_scale_data_management/blob/main/VoiD_with_PySpark_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Installing SPARK for Playing


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -q findspark
import findspark
findspark.init()


# SPARK Installed... Start playing

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
!wget -q https://storage.googleapis.com/public_lddm_data/small_page_links.nt
!ls


sample_data	     small_page_links.nt.1
small_page_links.nt  spark-3.3.0-bin-hadoop3.tgz


In [7]:
lines = spark.read.text("small_page_links.nt").rdd.map(lambda r: r[0])


In [8]:
lines.take(5)

['<http://dbpedia.org/resource/AfghanistanHistory> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/History_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AfghanistanGeography> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Geography_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AccessibleComputing> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Computer_accessibility> .',
 '<http://dbpedia.org/resource/AfghanistanPeople> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Demography_of_Afghanistan> .',
 '<http://dbpedia.org/resource/AfghanistanCommunications> <http://dbpedia.org/property/wikilink> <http://dbpedia.org/resource/Communications_in_Afghanistan> .']

In [9]:
lines.count()

100000

In [15]:
import re
def parseTriple(line) :
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', line)
    return parts[0], parts[1], parts[2]
triples=lines.map(lambda x:parseTriple(x))
triples.count()
triples.take(1)

[('<http://dbpedia.org/resource/AfghanistanHistory>',
  '<http://dbpedia.org/property/wikilink>',
  '<http://dbpedia.org/resource/History_of_Afghanistan>')]

# Exercice: Computing the VoiD for the Nt File with PySpark RDD

Write RDD PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

- QB1 CONSTRUCT { <D> v:triples ?x }
WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

- QB2 CONSTRUCT { <D> v:classes ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

- QB3 CONSTRUCT { <D> v:properties ?x }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

- QB4 CONSTRUCT { <D> v:distinctSubjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

- QB5 CONSTRUCT { <D> v:distinctObjects ?x }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [ ]:
your code ??

- QC1 CONSTRUCT { <D> v:classPartition [ v:class ?c ] } WHERE { ?s a ?c }

- QC2 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:triples ?x ] }
WHERE { SELECT (COUNT(?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }
- QC3 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:classes ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?d) AS ?x) ?c WHERE { ?s a ?c , ?d } GROUP BY ?c }
- QC4 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:properties ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c } 
- QC5 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c WHERE { ?s a ?c } GROUP BY ?c }
- QC6 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c }

In [ ]:
Your code ??

- QD1 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ] } WHERE { ?s ?p ?o }
- QD2 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:triples ?x ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD3 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctSubjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }
- QD4 CONSTRUCT { <D> v:propertyPartition [ v:property ?p ; v:distinctObjects ?x ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?p WHERE { ?s ?p ?o } GROUP BY ?p }

In [ ]:
Your code ??

- QE1 CONSTRUCT { <D> v:classPartition [ v:class ?c ; v:propertyPartition [ v:property ?p ] ] }
WHERE { ?s a ?c ; ?p ?o }
- QE2 CONSTRUCT { <D> v:classPartition [ v:class ?c
v:propertyPartition [ v:property ?p ; v:triples ?x ] ] }
WHERE { SELECT (COUNT(?o) AS ?x) ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE3 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctSubjects ?x ] ] }
WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }
- QE4 CONSTRUCT { <D> v:classPartition [ v:class ?c ;
v:propertyPartition [ v:distinctObjects ?x ; v:property ?p ] ] }
WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) ?c ?p WHERE { ?s a ?c ; ?p ?o } GROUP BY ?c ?p }

In [ ]:
Your code

# Same Exercice with PySpark SQL



In [23]:
from pyspark.sql import Row
spo=triples.map(lambda x: Row(subject=x[0],predicate=x[1],object=x[2]))
df=spark.createDataFrame(spo)
df.createOrReplaceTempView("SPO")
df.show()



+--------------------+--------------------+--------------------+
|             subject|           predicate|              object|
+--------------------+--------------------+--------------------+
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<http://dbpedia.o...|<http://dbpedia.o...|
|<http://dbpedia.o...|<ht

In [25]:

result=spark.sql("select subject from SPO")
result.show()

+--------------------+
|             subject|
+--------------------+
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
|<http://dbpedia.o...|
+--------------------+
only showing top 20 rows



Write SQL PySpark for the following queries (your can ignore the construct statement, just compute the select part...)

QB1 CONSTRUCT { v:triples ?x } WHERE { SELECT (COUNT(*) AS ?x) WHERE { ?s ?p ?o } }

QB2 CONSTRUCT { v:classes ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s a ?o } }

QB3 CONSTRUCT { v:properties ?x } WHERE { SELECT (COUNT(DISTINCT ?p) AS ?x) WHERE { ?s ?p ?o } }

QB4 CONSTRUCT { v:distinctSubjects ?x } WHERE { SELECT (COUNT(DISTINCT ?s) AS ?x) WHERE { ?s ?p ?o } }

QB5 CONSTRUCT { v:distinctObjects ?x } WHERE { SELECT (COUNT(DISTINCT ?o) AS ?x) WHERE { ?s ?p ?o } }

In [ ]:
Your Code